In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29501
2,Huelva,Confirmados PDIA 14 días,1125
3,Huelva,Tasa PDIA 14 días,"219,22559775512988"
4,Huelva,Confirmados PDIA 7 días,615
5,Huelva,Total Confirmados,29677
6,Huelva,Curados,26297
7,Huelva,Fallecidos,370


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29501.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7708.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 749 personas en los últimos 7 días 

Un positivo PCR cada 418 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29501.0,615.0,1125.0,513170.0,119.843327,219.225598,251.0
Condado-Campiña,9422.0,268.0,507.0,156231.0,171.540859,324.519462,118.0
Huelva-Costa,17123.0,302.0,538.0,289548.0,104.300496,185.806844,108.0
Huelva (capital),7708.0,192.0,344.0,143837.0,133.484430,239.159604,73.0
Sierra de Huelva-Andévalo Central,2615.0,45.0,77.0,67391.0,66.774495,114.258581,26.0
Almonte,1653.0,81.0,159.0,24507.0,330.517811,648.794222,24.0
Moguer,1153.0,37.0,63.0,21867.0,169.204738,288.105364,22.0
Punta Umbría,754.0,37.0,44.0,15355.0,240.963855,286.551612,18.0
Bollullos Par del Condado,893.0,43.0,91.0,14387.0,298.880934,632.515465,17.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Galaroza,39.0,9.0,12.0,1382.0,651.230101,868.306802,2.0
Almonte,1653.0,81.0,159.0,24507.0,330.517811,648.794222,24.0
Bollullos Par del Condado,893.0,43.0,91.0,14387.0,298.880934,632.515465,17.0
Escacena del Campo,153.0,6.0,9.0,2287.0,262.352427,393.528640,4.0
Cortelazor,8.0,1.0,1.0,299.0,334.448161,334.448161,0.0
Rociana del Condado,533.0,12.0,26.0,7939.0,151.152538,327.497166,4.0
Condado-Campiña,9422.0,268.0,507.0,156231.0,171.540859,324.519462,118.0
Nerva,169.0,9.0,16.0,5169.0,174.114916,309.537628,7.0
Niebla,205.0,9.0,12.0,4116.0,218.658892,291.545190,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Palos de la Frontera,603.0,6.0,26.0,11742.0,51.098620,221.427355,4.0,0.230769
Ayamonte,1296.0,5.0,19.0,21104.0,23.692191,90.030326,0.0,0.263158
Beas,203.0,2.0,7.0,4341.0,46.072334,161.253167,1.0,0.285714
Bonares,259.0,2.0,7.0,6060.0,33.003300,115.511551,0.0,0.285714
Cartaya,1292.0,10.0,31.0,20083.0,49.793358,154.359408,1.0,0.322581
Lucena del Puerto,236.0,3.0,9.0,3261.0,91.996320,275.988960,2.0,0.333333
Aljaraque,1020.0,12.0,27.0,21474.0,55.881531,125.733445,4.0,0.444444
Rociana del Condado,533.0,12.0,26.0,7939.0,151.152538,327.497166,4.0,0.461538
Bollullos Par del Condado,893.0,43.0,91.0,14387.0,298.880934,632.515465,17.0,0.472527
